# 파일 인코딩 변경 프로그램

* 내 컴퓨터의 ANSI 포맷 파일을 UTF-8로 변경해주는 프로그램
* py파일 실행시 인자값을 받아서 대상 폴더의 모든 파일과 하위폴더의 조건에 맞는 파일을 모두 변경

* 난이도 : ★★☆☆☆☆☆☆☆☆
* 다루는 내용:
    * 재귀함수, 실행시 인자값 넘기기, 문자 인코딩 알아내기


* 참고내용:
    * 한글 윈도우 메모장에서의 인코딩방식은
    * ANSI: 기본적인 한글은 cp949
    * 유니코드: UTF-16
    * 유니코드(big endian) : UTF-16
    * UTF-8: UTF-8-SIG

In [ ]:
import os                   # file 관련 함수들을 사용하기 위해 사용
import argparse             # py 파일 실행시 실행인자값을 받기 위해 사용
from chardet import detect  # 파일의 문자열 인코딩 방식을 알기 위해 사용

# 인코딩방식을 변경할 대상 확장자
INCLUDE_EXT_LISTS = [".txt", ".smi"]

def search_dir(dirname):
    '''대상 폴더 내의 파일과 서브 폴더를 탐색하는 재귀 함수

    Args:
        dirname (str): 탐색 대상 폴더 경로

    Returns:
        list : 탐색한 결과 파일 리스트를 리턴
    '''
    # 결과를 리턴할 리스트 변수
    result_file_lists = []

    # dirname 의 경로의 파일과 폴더 목록을 구함
    filenames = os.listdir(dirname)

    # 파일과 폴더 목록 반복문
    for filename in filenames:
        # filename 에는 os.listdir(dirname) 에서 dirname 이후의 경로값과 파일명만 존재하기 때문에
        # 전체 풀 경로를 얻기 위해선 dirname 과 filename 을 합쳐야 함
        full_filepath = os.path.join(dirname, filename)

        # full_filepath 가 디렉토리라면..
        if os.path.isdir(full_filepath):
            # search_dir 인 자기 자신 함수를 다시 호출한다.(재귀함수)
            result_file_lists.extend(search_dir(full_filepath))
        # 파일인 경우 result_file_lists 에 추가
        else:
            result_file_lists.append(full_filepath)

    # 결과 목록 리턴
    return result_file_lists

def get_encoding_type(file):
    '''파일의 인코딩 타입을 구하는 함수

    Args:
        file (str): 파일 경로

    Returns:
        str : 인코딩 타입(ascii, utf-8-sig... 등)
    '''
    # 파일을 rb 모드로 읽기한다.
    with open(file, 'rb') as f:
        rawdata = f.read()
    # 파일내용으로 detect 함수를 통해 인코딩 타입을 유추하여 리턴
    return detect(rawdata)['encoding']


if __name__ == "__main__":
    # 실행시 인자값을 편하게 사용하기 위해 argsparse 사용
    parser = argparse.ArgumentParser()

    # 어떤 아규먼트를 사용할지 등록
    # -f 는 대상 폴더 인자, str 형
    parser.add_argument("-f", type=str, help="[대상 폴더명]")
    
    # -e 는 대상 확장자 인자인데 nargs="+" 는 -e smi txt py cpp 이런식으로 열거식으로 받아서 리스트 형태로 사용됨
    # nargs 대신 action='append' 도 사용할 수 있는데 append 는 -e .smi -e .txt 이렇게 반복해서 인자를 써야함
    parser.add_argument("-e", nargs="+", help="[대상 확장자]")

    # 파서를 설정완료하고 args 변수에 할당
    # 실제 실행시 넘어온 인자는 args 로 컨트롤
    args = parser.parse_args()

    # f 대상 폴더 인자가 없는 경우에는 사용법을 안내해줌
    if args.f is None:
        print("=" * 90)
        print("사용법: python " + str(os.path.abspath(__file__)).split("\\")[-1] + " -f [대상폴더명] -e .txt .smi .... [대상 확장자]")
        print("=" * 90)
    else:
        # 실행 인자로 넘어온 대상 폴더 (args.f) 를 검색함
        filelists = search_dir(args.f)

        # 실행인자로 대상 확장자 값이 넘어왔다면
        if args.e is not None:
            if len(args.e) > 0:
                # args.e 리스트의 요소에 '.' 이 있나 없나 확인해서 없으면 '.' 을 붙인 후 소문자로 치환해서 INCLUDE_EXT_LIST에 할당
                INCLUDE_EXT_LISTS = [e.lower() if e[0:1] == "." else ".{}".format(e.lower()) for e in args.e]
                print(INCLUDE_EXT_LISTS)

        # 대상폴더의 파일목록을 반복
        for filepath in filelists:
            # 전체 경로에서 확장자와 나머지를 분리
            file, ext = os.path.splitext(filepath)

            # 경로/파일명 + _tmp + 확장자로 임시 파일명 생성(파일명만 생성한거임)
            tempfile = file + "_tmp" + ext

            # 대상확장자인지 확인
            if ext.lower() in INCLUDE_EXT_LISTS:
                # 파일의 인코딩 타입을 구함
                encoding = str(get_encoding_type(filepath)).lower()

                # 인코딩 타입에 utf 가 없다면 (utf-8, utf-16, utf-8-sig..) 변환 대상
                if encoding.find("utf") < 0:
                    # 인코딩 오류를 방지하기 위해 try / except
                    try:
                        # 원본파일은 읽기 모드, 임시파일은 쓰기 모드로 열어서 오픈후 쓰기
                        # 여기서 쓰기파일은 utf-8-sig 로 인코딩을 주었음
                        # 윈도우에서는 기본적인 UTF-8 형식에 BOM signature가 자동으로 들어가기 때문에 utf-8-sig 로 해야함
                        # BOM = Byte Order Mark 로 문서 첫 바이트에 어떤 인코딩으로 된 파일인지 정확하게 마킹하는것을 말함
                        # UTF-8: EF BB BF
                        # UTF-16 Big Endian: FE FF
                        # UTF-16 Little Endian: FF FE
                        # UTF-32 Big Endian: 00 00 FE FF
                        # UTF-32 Little Endian: FF FE 00 00
                        # 아~ 그렇구만 하고 넘어가면 됨!
                        with open(filepath, "r") as read, open(tempfile, "w", encoding="utf_8_sig") as write:
                            write.write(read.read())

                        # 원본 파일 삭제
                        os.unlink(filepath)

                        # 임시파일명을 원본 파일명으로 변경
                        os.rename(tempfile, filepath)

                        # 결과 출력
                        print("{} 파일 인코딩 변경".format(filepath))
                    except Exception as e:
                        # pass 는 아무것도 안쓰면 오류나기 때문에 아무 내용이 없을때 오류를 방지하기 위해 씀
                        pass
                    finally:
                        # 최종적으로 오류가 났던 안났던 만약 임시 파일이 존재하면 삭제
                        if os.path.exists(tempfile):
                            os.unlink(tempfile)
